<a href="https://colab.research.google.com/github/minsing-jin/Menu_reccommendation/blob/main/menu_recommendation_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MENU recommendation program

- 자취 예산을 맞춰서 자동으로 2주치 식단표를 짜주는 프로그램입니다

- 예산을 입력하면 쿠팡 식재료 최저가 기준으로 끼니마다의 메뉴들을 예산 아래로 맞춰주어 자동으로 식단표를 추천해줍니다.

- 자취 기준이므로 빨리 해먹고 빨리 치울수 있는 것으로 하는 메뉴로 선정했습니다.

# Motivation
- koprivategpt 프로젝트를 진행하기 위해서 맥북 프로 구입으로 인한 갑작스러운 가난 이슈

- 자취밥값 계산하려다가 급 귀찮아짐 issue(하나하나 다 계산하는거 너무 귀찮아유)

# 필요한것
- 메뉴당 얼마가 들지 항시 업데이트 or 프로그램 돌릴때마다 업데이트(csv파일 수작업)
> 메뉴당 어떠한 식재료가 필요한가 초반 csv 파일 작업 후 pandas로 불러온다음 메뉴의 값은 프로그램에서 돌리면서 업데이트

 > 메뉴판 구성할때 categorize를 하여 식단표 구성할때 옵션을 구성


- 쿠팡 식재료 종류 자동검색 및 최저가에 해당하는 식재료 크롤링 후 선별

- 링크 포함

- 식단표에 해당하는 table 자동 작성 기능

- 시간 가용한대로 slack bot에 전달

- 한식일경우 메인메뉴와 반찬의 조합을 따질것

### 검색 조건 및 선택지

- 반찬 완제품으로 할것인가?

- 예산은 얼마나 쓸것인가?

- 밀키트를 얼마나 쓸것인가?

- 양식, 중식, 한식, 일식 비율

### Issue

- 양념과 같은 것은 어떻게 할것인가?(한번 사면 꽤 오래 쓰는 것들)
>3달에 한번씩 사는걸로 하고 시세만 계산해주기<br>
그 시세를 3으로 나눈후(1달당 소비되는 양에 대한 양념가격) 예산입력후 미리 차감하여 메뉴 구성

- 식재료마다 소진되는 속도차이는 어떻게 해결할것인가?

- 돼지고기 같은 소분해야하는 식재료들은 어떻게 분배할것인가? -> 조건설정

- 식재료 한번 사면 얼마나 분배? 식재료 특성들 다 마킹해서 몇번 쓸 수 있을지 점수로 분배?

- 기본 템들은 어떻게 할것? 매번 살수는 없음

default item
1. 밥(한달로 계산)
2. 간장
3. 깨
4. 소금
5.


- 반찬은 따로 구입(너무 시간 많이 듦)

- 아니면 재료마다 쿠팡 최저가 검색후
가격에 쓸수 있는 횟수를 나눠서 1회 평균 사용가격으로 예산 맞추기?

- 장보기 recommendation

- 음식하는 시간은 최대한 줄일 수 있도록 시간을 벌 수 있는 메뉴로 구성하기

- 아예 밀키트로 구성할까?

- 일주일 3일 닭가슴살



# 메뉴판 업데이트

## CSV파일 불러오기

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd

Korean_menu = pd.read_csv('./drive/MyDrive/menu/Korean_main.csv').iloc[:,1:]
American_menu = pd.read_csv('./drive/MyDrive/menu/American_main.csv').iloc[:,1:]


In [18]:
print(Korean_menu)

          제육볶음 간장계란밥 비엔나 소시지 스팸김치볶음밥   돼지고기김치찌개 참치김치찌개  계란찜   된장찌개  비빔국수  \
0   돼지고기(600g)     밥     비엔나      김치  돼지고기 앞다리살    참치캔   계란     양파    국수   
1           양파    계란      양파      대파         김치     김치    파  느타리버섯    깻잎   
2         청양고추   NaN    파프리카      스팸          파     대파  NaN    애호박    오이   
3           대파   NaN     NaN      계란         두부   청양고추  NaN     두부    계란   
4          NaN   NaN     NaN     NaN        NaN    NaN  NaN   팽이버섯   골뱅이   
5          NaN   NaN     NaN     NaN        NaN    NaN  NaN      파   NaN   
6          NaN   NaN     NaN     NaN        NaN    NaN  NaN    NaN   NaN   
7          NaN   NaN     NaN     NaN        NaN    NaN  NaN    NaN   NaN   
8          NaN   NaN     NaN     NaN        NaN    NaN  NaN    NaN   NaN   
9          NaN   NaN     NaN     NaN        NaN    NaN  NaN    NaN   NaN   
10         NaN   NaN     NaN     NaN        NaN    NaN  NaN    NaN   NaN   
11         NaN   NaN     NaN     NaN        NaN    NaN  NaN    NaN   NaN   
12         N

## Data preprocessing

### Default ingridents classifiation

In [31]:
tmp = Korean_menu.loc[18:].reset_index(drop=True)
# print(tmp)
# print(tmp["Unnamed: 0"])
default_ingrents = []
cnt = 0
print(len(tmp["제육볶음"]))
for i in tmp:
  print(tmp[i][cnt])
  default_ingrents.append(tmp[i])
  cnt += 1

# print(default_ingrents)


9


In [ ]:
ㅡ